# MCMC1.0: Introduction to Markov Chain Monte Carlo

The aim of the Markov chain Monte Carlo (MCMC) is to produce random variables obeying any probability distributions you like. There are three main tools for MCMC and I will introduce them one by one.

* Metropolis-Hastings algorithm
* Gibbs sampler
* Hybrid Monte Carlo

## Metropolis-(Hastings) algorithm

The Metropolis-Hastings (MH) algorithm is the most important and most popular tool in MCMC, Bayesian statistics, and physics simulations. It is universal and applicable to any kinds of distributions. The other methods usually have some restrictions, so if you do not know how to use another one like a Gibbs sampler, you should choose the MH method.

### Classical Monte Carlo simulation for Ising model

As a typical example, I will present a simple Julia implimentation of the Monte Carlo simulation of the 2D Ising model on the square lattice. The Metropolis rule is simply given by the following function.

In [1]:
function Metropolis(β::Float64, ΔE::Float64)::Bool
    (ΔE <= 0.0) || (exp(-β * ΔE) > rand())
end

Metropolis (generic function with 1 method)

This function will tell us whether we will accept a new state or not according to its energy gain $\Delta E$. It is clear that acceptance/rejection will be determined probabilistically and typically the MH alrorithm has an acceptance rate of only 20-40% (c.f. in the Ising model it is much higher). I will later show you algorithms with an almost 100% acceptance rate, such as a Gibbs sampler or a hybrid Monte Carlo.

### Storing spin configurations

The most memory-effcient way to store a spin configuration is using bitwise operations. Usually we need a very low-level code for such bitwise operations (see e.g. "Hacker's Delight" by H. S. Warren), but Julia provides a nice platform called BitArray. We regard a bit 1 as spin-up ($\sigma =+1$) and 0 as spin-down ($\sigma =-1$). For 1D, we can just store an array of spins as BitArray.

In [2]:
L = 8
spin1D = trues(L)

8-element BitArray{1}:
 true
 true
 true
 true
 true
 true
 true
 true

In order to store a 2D spin configuration, Array of BitArray is more useful than 2D BitArray (you will understand later).

In [3]:
spin2D = rand(Bool, L, L)
spin2Drow = [BitVector(spin2D[n, :]) for n in 1 : L]

8-element Array{BitArray{1},1}:
 [true, false, false, true, true, true, true, true]   
 [false, true, true, true, true, true, true, false]   
 [true, true, false, false, true, true, false, false] 
 [true, false, false, true, false, false, true, false]
 [true, true, true, true, false, true, false, true]   
 [false, true, true, true, true, true, true, false]   
 [true, false, true, true, false, true, true, true]   
 [false, true, true, true, false, true, true, false]  

For the future purpose, it is useful later to store the transposed version:

In [4]:
spin2Dcolumn = [BitVector(spin2D[:, n]) for n in 1 : L]

8-element Array{BitArray{1},1}:
 [true, false, true, true, true, false, true, false]  
 [false, true, true, false, true, true, false, true]  
 [false, true, false, false, true, true, true, true]  
 [true, true, false, true, true, true, true, true]    
 [true, true, true, false, false, true, false, false] 
 [true, true, true, false, true, true, true, true]    
 [true, true, false, true, false, true, true, true]   
 [true, false, false, false, true, false, true, false]

### Updating

Assuming a periodic boundary condition, the nearest-neighbor sites can be obtained by bitwise shift (rotation) operations. It is appropriate for the small $L$ but for a larger size it has a large overhead.

In [5]:
J = -1.0 # Ferro
function energygain(x::Int64, y::Int64, row::Vector{BitVector}, column::Vector{BitVector})::Float64
    left = xor(circshift(row[y], 1)[x], row[y][x])
    right = xor(circshift(row[y], -1)[x], row[y][x])
    top = xor(circshift(column[x], 1)[y], column[x][y])
    bottom = xor(circshift(column[x], -1)[y], column[x][y])
    -2. * J * sum(@. 1 - 2 * [left, right, top, bottom]) # This is not a smart way
end
energygain(3, 1, spin2Drow, spin2Dcolumn)

-4.0

If we choose the flipping spin sequentially (sequential sweep), it may destroy a detailed balance microscopically. Usually it is better to choose a candidate spin using a random variable. Note that functions can be used as arguments in Juila.

In [6]:
function step!(β::Float64, method::Function, row::Vector{BitVector}, column::Vector{BitVector})::Tuple
    x = rand(1 : L)
    y = rand(1 : L)
    ΔE = energygain(x, y, row, column)
    flip = method(β, ΔE)
    row[y][x] = xor(row[y][x], flip)
    column[x][y] = xor(column[x][y], flip)
    x, y, flip # Actually, it is not recommeded for destructive functions to return variables 
end
step!(0.1, Metropolis, spin2Drow, spin2Dcolumn)

(6, 5, true)

Julia is using "call by reference" for Array, so spin2Drow will be directely updated.

In [7]:
spin2Drow

8-element Array{BitArray{1},1}:
 [true, false, false, true, true, true, true, true]   
 [false, true, true, true, true, true, true, false]   
 [true, true, false, false, true, true, false, false] 
 [true, false, false, true, false, false, true, false]
 [true, true, true, true, false, false, false, true]  
 [false, true, true, true, true, true, true, false]   
 [true, false, true, true, false, true, true, true]   
 [false, true, true, true, false, true, true, false]  

### Simulations

Let's warm up!

In [8]:
function warmup!(β::Float64, spin2Drow::Vector{BitVector}, spin2Dcolumn::Vector{BitVector})
    N = 100000
    for i in 1 : N
        step!(β, Metropolis, spin2Drow, spin2Dcolumn)
    end
end
warmup!(0.1, spin2Drow, spin2Dcolumn)
spin2Drow

8-element Array{BitArray{1},1}:
 [false, true, false, false, false, true, false, false]
 [true, false, false, true, true, true, false, true]   
 [true, false, false, true, false, false, false, true] 
 [true, false, false, false, true, true, false, false] 
 [false, true, false, true, true, false, true, true]   
 [false, false, true, false, true, false, false, false]
 [false, true, false, false, false, false, true, true] 
 [false, true, true, false, false, true, true, false]  

At low temperature, the spins will be ordered to the ferromagnetic state.

In [9]:
warmup!(10.0, spin2Drow, spin2Dcolumn)
spin2Drow

8-element Array{BitArray{1},1}:
 [false, false, false, false, false, false, false, false]
 [false, false, false, false, false, false, false, false]
 [false, false, false, false, false, false, false, false]
 [false, false, false, false, false, false, false, false]
 [false, false, false, false, false, false, false, false]
 [false, false, false, false, false, false, false, false]
 [false, false, false, false, false, false, false, false]
 [false, false, false, false, false, false, false, false]

## Gibbs sampler (heat-bath method)

A Gibbs sampler is the most powerful tool in MCMC if you are able to generate a "conditional probability distribution." Only in this case, we can achieve a 100% acceptance rate for the continuous variable distributions, i.e. the acceptance/rejection process is deterministic. For some reason a Gibbs sampler is called heat-bath method in physics, but you should learn it in this generalized form.

### Example

Assuming we want to sample from a distribution defined by a probablity
$$
p(x,y) \propto e^\frac{-a x^2+2b xy - cy^2}{2},
$$
we can explicitly derive a conditional distribution as a Gaussian distribution for each variable.
$$
p(x|y) \propto e^{-\frac{a}{2} \left(x - \frac{b}{a}y\right)^2}, \\
p(y|x) \propto e^{-\frac{c}{2} \left(y - \frac{b}{c}x\right)^2}.
$$

By updating $x$ and $y$ alternately from these two conditional distributions, we can produce the original joint distribution because it is the invariant distribution of these two sampling processes. Note that the detailed balance is microscopically broken, but the algorithm is balanced as a total. If you wish to resotre it, it is better to choose $x$ or $y$ randomly.

In [10]:
using Distributions
function Gibbssampler(a::Float64, b::Float64, c::Float64, x::Float64, y::Float64)::Tuple{Float64, Float64}
    x = rand(Normal(b / a * y, 1 / sqrt(a)))
    y = rand(Normal(b / c * x, 1 / sqrt(c)))
    x, y
end
Gibbssampler(1.0, 0.8, 1.0, 0.0, 0.0)

(-1.239921547930485, -0.5389071016521685)

Julia includes the library for various distributions, so the implementation should be simple. After the convergence it will give us a correct distribution.

In [11]:
function sampling(a::Float64, b::Float64, c::Float64)::Tuple{Float64, Float64}
    N = 100000
    x = 0.0
    y = 0.0
    for i in 1 : N
        x, y = Gibbssampler(a, b, c, x, y)
    end
    x, y
end
sampling(1.0, 0.8, 1.0)

(-1.4499480181974609, -0.5149621718441911)

### For Ising model

For the Ising model, the Gibbs sampler will reduce to just a so-called heat-bath method, which has been named probably because the unupdated spins can be regarded as a heat bath and the transition probablities obey the partial "canonical distribution" for the updated spin. The implementation is simple and can be done just by replacing the function Metropolis to the following one:

In [12]:
function heatbath(β::Float64,ΔE::Float64)::Bool
    (1.0 / (exp(β * ΔE) + 1.0)) > rand()
end
step!(0.1, heatbath, spin2Drow, spin2Dcolumn)

(7, 1, true)

Note that the acceptance rate is apparently not 100% due to the finite number of the possible transitions.

In [13]:
function heatup!(β::Float64, spin2Drow::Vector{BitVector}, spin2Dcolumn::Vector{BitVector})
    N = 100000
    for i in 1 : N
        step!(β, heatbath, spin2Drow, spin2Dcolumn)
    end
end
heatup!(0.1, spin2Drow, spin2Dcolumn)
spin2Drow

8-element Array{BitArray{1},1}:
 [true, false, true, false, false, true, true, false]  
 [true, false, true, false, false, false, false, false]
 [true, true, false, false, false, false, false, false]
 [true, false, true, false, false, true, false, true]  
 [false, false, true, false, true, true, true, true]   
 [true, false, false, false, true, true, true, true]   
 [true, true, true, false, true, false, false, false]  
 [false, true, true, true, true, true, true, true]     

In [14]:
heatup!(10.0, spin2Drow, spin2Dcolumn)
spin2Drow

8-element Array{BitArray{1},1}:
 [true, true, true, true, true, true, true, true]
 [true, true, true, true, true, true, true, true]
 [true, true, true, true, true, true, true, true]
 [true, true, true, true, true, true, true, true]
 [true, true, true, true, true, true, true, true]
 [true, true, true, true, true, true, true, true]
 [true, true, true, true, true, true, true, true]
 [true, true, true, true, true, true, true, true]

### Blocked Gibbs sampler

From now on, I will introduce some variants of the original Gibbs sampler. If we have 3 or more parameters for the Gibbs sampler, then we can group them into 2 or more groups.

~ under construction ~

### Collapsed Gibbs sampler

We can also integrate out some parameter to get a collapsed Gibbs sampler.

~ under construction ~

## Hybrid Monte Carlo

Condensed matter physicists do not usually use this method because they are mostly using discrete variables, but this method is the most efficient method with an almost 100% acceptance rate "for continuous variables" because it is using a gradient of the probability distribution. A hybrid Monte Carlo or Hamiltonian Monte Carlo (HMC) would be the best solution for the distribution with a known gradient and it is much more efficient than the pure MH method because of the highest acceptance rate.

The essence of HMC can be understood as some hybrid of the MH method and the Gibbs sampler (or deterministic molecular dynamics), and HMC generates an extended probability distribution including an auxiliary field called "momentum," where the conditional probability distribution without this momentum becomes the disired one. In some quantum Monte Carlo (QMC) simulations, it is difficult to calculate the gradient explicitly, and then we have to introduce another auxiliary field (called "pseudofermion" although it is actually bosonic) to estimate the gradient probabilistically "as an avarage."

### How it works

Instead of generating the original distribution $\exp(-V(\chi))$ (e.g. $V(\chi) = \beta E(\chi)$ for the continuous variables $\chi$), we would like to generate a joint distribution of $\chi$ and $\pi$ (if $\chi$ is a vector, $\pi$ should have the same dimention):
$$p(\chi,\pi)=\exp(-\pi^2-V(\chi)).$$
If we simply discard the auxiliary variables $\pi$, the resulting conditional distribution simply becomes the original one.
$$p(\chi | \pi)=\exp(-V(\chi)).$$
Thus it is enough to generate the extended distribution with doubled variables.

If we define a classical Hamiltonian as $\mathcal{H}=\pi^2+V(\chi)$, the problem results in the generation of the distribution of the classical probabilistic system with $p=\exp(-\mathcal{H})$. Note that we simply set the mass $m=1/2$, but we can generalize it to any mass matrix.

### Algorithm

The updating process goes similarly to the Gibbs sampler. First, we update $\pi$ from
$$p(\pi | \chi)=\exp(-\pi^2).$$
This is just a normal distribution, so Box–Muller's method simply works for this step.

We use a deterministic molecular dynamics to update $\chi$. Since the equation of motion conserves the total energy, the distribution is not changed by this motion from $t=0$ to any time $t=T$ when we update both $\chi$ and $\pi$ simultaneously according to the canonical equation. Typically, we move $(\chi,\pi)$ for $T=1$ according to the randomly selected momentum $\pi$ in order to move $\chi$ for a long enough distance (to avoid the self-correlation). The detailed balance completely holds for this deterministic updating *as long as the equation of motion is solved exactly without any numerical error.* Under this (numerically impossible) assumption, we can achieve 100% acceptance rate for this HMC algorithm.

### Leap-flogging

In the ideal case with a negligible numerical error, HMC can achieve almost 100% acceptance rate and in this sense it is very efficient, but this is impossible in a real computer using floating-point numbers. Thus, we use the following integrator for the equation of motion to keep its detailed balance:

1. $\chi(t+\Delta t/2)  = \chi(t) + \pi(t) \Delta t.$
1. $\pi(t+ \Delta t) = \pi(t) - \nabla V(\chi+\Delta t/2) \Delta t.$
1. $\chi(t+\Delta t)  = \chi(t+\Delta t/2) + \pi(t+ \Delta t) \Delta t.$

This algorithm is apparently reversible, i.e. everything goes back when we replace $\pi \to -\pi$. This fact will not be affected by the numerical error because the unupdated variable is conserved for each step (see e.g. the lifting algorithm for the wavelet transformation). Thus, the detailed balance (i.e. reversibility of the process) is still kept even with a numerical error for this leap-flogging approximation considering the fact that the normal distribution produces the same amount of $\pi$ and $-\pi$.

However, if $\Delta t$ is not small enough, the energy $\mathcal{H}$ is no longer conserved due to the numerical error and we need an acceptance/rejection process to keep its distribution. This is simply realized by the MH algorithm for the accumulated error for the energy $\Delta \mathcal{H} = \mathcal{H}_\textrm{new} - \mathcal{H}_\textrm{old}$. Note that we are dealing with a probability density, not a probability itself, so the measure $d\chi d\pi$ of $p(\chi,\pi)d\chi d\pi$ also has to be conserved. The leap-flogging has no problem on this point because it clearly conserves the symplectic form $d\chi d\pi$ regardless of the error (take the exact differential of each step).

If we begin from the MH algorithm (other way around), HMC can be regarded as an improved version of the random walk + MH algorithm. If we just do a random walk (with a momentum $\pi$) and check it by MH, there is no reason to think that the energy difference $\Delta \mathcal{H} = \mathcal{H}_\textrm{new} - \mathcal{H}_\textrm{old}$ becomes small. However, if we use a molecular dynamics instead of the random walk, the origin of $\Delta \mathcal{H}$ is totally from the numerical error and we can expect that it becomes small if the calculation is accurate enough. HMC is one simple way to increase the acceptance rate of the MH algorithm to almost 100%.

### Implementation

In [15]:
function leapflogging(χ, π, Δt::Float64, N::Int64, ∇V::Function)::Tuple
    Δχ = π * Δt
    χ += Δχ
    for i in 1 : (N - 1)
        π -= ∇V(χ) * Δt
        χ += 2Δχ
    end
    π -= ∇V(χ) * Δt
    χ += Δχ
    χ, π
end

leapflogging (generic function with 1 method)

In [17]:
using LinearAlgebra
function HMC(χ::Vector{Float64}, V::Function, ∇V::Function)::Vector{Float64}
    Nstep = 10000
    Nintegrate = 10
    Δt = 1 / Nintegrate
    π = similar(χ) # Actually, it is not good to use π=3.14...
    for i in 1 : Nstep
        π .= rand(Normal(0., 1. / sqrt(2.)), length(π))
        Hold = π ⋅ π + V(χ)
        χold = χ
        πold = π
        χ, π = leapflogging(χ, π, Δt, Nintegrate, ∇V)
        Hnew = dot(π, π) + V(χ)
        #println((Hold, Hnew))
        if Metropolis(1., Hnew - Hold)
            #println("Accepted!")
        else
            χ = χold
            π = πold
        end
    end
    χ
end
v = x -> (x[1] ^ 2 - 2 * 0.8x[1] * x[2] + x[2]^2) / 2.
dv = x -> [x[1] - 0.8x[2], x[2] - 0.8x[1]]
HMC([0.0, 0.0], v, dv)

2-element Array{Float64,1}:
 1.531460989643879 
 3.3685009236667263

### Automatic Differentiation

Now in the field of machine learning or Bayesian statistics, the "autograd" i.e. automatic differentiation becomes one of the most powerful tools. Physicists still like to do differentiations by themselves, but a functional language like Julia can differentiate the function directly, so we no longer need any paper/pen.

In [18]:
using ForwardDiff
HMC([0.0, 0.0], v, x -> ForwardDiff.gradient(v, x))

2-element Array{Float64,1}:
 5.468502072637202 
 1.6963115723574302

See https://github.com/JuliaDiff/ForwardDiff.jl